<a href="https://colab.research.google.com/github/aryanasadianuoit/Adaptive-Graph-Based-Cohort-Creation-For-Deep-MutualLearning/blob/master/KD_Different_Methods.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt 
import numpy as np
from tensorflow import keras
from tensorflow.keras.models import Sequential, load_model, Model
from tensorflow.keras.layers import Conv2D,GlobalAveragePooling2D,Dense,Softmax,Flatten,MaxPooling2D,Dropout,Activation, Lambda, concatenate
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.losses import kullback_leibler_divergence as KLD_Loss, categorical_crossentropy as logloss
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.metrics import categorical_accuracy
import seaborn as sns

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
NUM_CLASSES = 10
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
print("x_train shape:", x_train.shape, "y_train shape:", y_train.shape)

# Normalize the dataset
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

170500096/170498071 [==============================] - 6s 0us/step
x_train shape: (50000, 32, 32, 3) y_train shape: (50000, 1)


In [6]:
def teacher_generator():
  teacher = Sequential() # Must define the input shape in the first layer of the neural network
  teacher.add(Conv2D(filters=32, kernel_size=2, padding='same', activation='relu', input_shape=(32,32,3)))
  teacher.add(MaxPooling2D(pool_size=2))
  teacher.add(Conv2D(filters=64, kernel_size=2, padding='same', activation='relu'))
  teacher.add(MaxPooling2D(pool_size=2))
  teacher.add(Flatten())
  teacher.add(Dense(256, activation='relu'))
  teacher.add(Dense(64, activation='relu',name="teacher_target_layer"))
  teacher.add(Dropout(0.5))
  teacher.add(Dense(10))
  teacher.add(Activation('softmax'))

  teacher.compile(loss='sparse_categorical_crossentropy',
             optimizer='adam',
             metrics=['accuracy'])

  # Take a look at the model summary

  #teacher.summary()

  return teacher

In [9]:
def student_generator():
  student = Sequential() #a Must define the input shape in the first layer of the neural network
  student.add(Flatten(input_shape=(32,32,3)))
  student.add(Dense(64, activation='relu'))
  student.add(Dense(10))
  return student

In [14]:
# Instanciating a teacher model and training it ===> for




#                                                                 OFFLINE KNOWLEDGE DISTILLATION


teacher_model = teacher_generator()
myCP = ModelCheckpoint(save_best_only=True,filepath='teacher_model.h5',monitor = 'val_acc')
teacher_model.fit(x_train,
         y_train,
         batch_size=128,
         epochs=20,
         validation_split = 0.2,
         callbacks=[myCP])

Epoch 1/20
313/313 [==============================] - 4s 13ms/step - loss: 1.7620 - accuracy: 0.3517 - val_loss: 1.3774 - val_accuracy: 0.5195
Epoch 2/20
313/313 [==============================] - 4s 12ms/step - loss: 1.3840 - accuracy: 0.5052 - val_loss: 1.1682 - val_accuracy: 0.5939
Epoch 3/20
313/313 [==============================] - 4s 11ms/step - loss: 1.2200 - accuracy: 0.5698 - val_loss: 1.0764 - val_accuracy: 0.6243
Epoch 4/20
313/313 [==============================] - 4s 11ms/step - loss: 1.1054 - accuracy: 0.6130 - val_loss: 1.0117 - val_accuracy: 0.6394
Epoch 5/20
313/313 [==============================] - 4s 12ms/step - loss: 1.0383 - accuracy: 0.6424 - val_loss: 0.9645 - val_accuracy: 0.6605
Epoch 6/20
313/313 [==============================] - 4s 12ms/step - loss: 0.9561 - accuracy: 0.6705 - val_loss: 0.9177 - val_accuracy: 0.6801
Epoch 7/20
313/313 [==============================] - 4s 12ms/step - loss: 0.8929 - accuracy: 0.6931 - val_loss: 0.9193 - val_accuracy: 0.6812

In [16]:
#                                                                OFFLINE KNOWLEDGE DISTILLATION

# Phase 1 ===>  re-create the TEACHER model with the softened softmax layer , Using the pre-defined TEMPERATURE

teacher_logits_model = Model(teacher_model.input,teacher_model.layers[-2].output)

Temperature = 3.25
T_layer = Lambda(lambda x:x/Temperature)(teacher_logits_model.output)
Softmax_layer = Activation('softmax')(T_layer)
teacher_soften_model = Model(teacher_model.input,Softmax_layer)
     

In [17]:
#                                                                OFFLINE KNOWLEDGE DISTILLATION

# Phase 2 ===>  Updating the training lables by adding the softened probabilities produced by the pre-trained TEACHER


# Predict and convert to sparse categorical matrix
y_train_new = teacher_soften_model.predict(x_train)
y_test_new = teacher_soften_model.predict(x_test)

y_train_new = np.c_[to_categorical(y_train),y_train_new]
y_test_new = np.c_[to_categorical(y_test),y_test_new]

In [19]:
#                                                                OFFLINE KNOWLEDGE DISTILLATION

# Phase 3 ===>  Instanciating a STUDENT. In default, the student_generator, generates student models that they do not have ACTIVATION in their last layer.
# After initializing the student model, we compute the logits, and add the last layer with of SOFTMAX


student_model =  student_generator()
student_model.summary()

student_logits = student_model.layers[-1].output

# Compute softmax
probs = Activation("softmax")(student_logits)

# Compute softmax with softened logits
logits_T = Lambda(lambda x:x/Temperature)(student_logits)
probs_T = Activation("softmax")(logits_T)

CombinedLayers = concatenate([probs,probs_T])

student_model_with_softmax = Model(student_model.input,CombinedLayers)


Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_9 (Flatten)          (None, 3072)              0         
_________________________________________________________________
dense_18 (Dense)             (None, 64)                196672    
_________________________________________________________________
dense_19 (Dense)             (None, 10)                650       
Total params: 197,322
Trainable params: 197,322
Non-trainable params: 0
_________________________________________________________________


In [20]:
#                                                                OFFLINE KNOWLEDGE DISTILLATION

# Phase 4 ===>  KNOWLEDGE DISTILLATION custom LOSS

def KD_loss(y_true,y_pred,lambd=0.5,T=10.0):
  y_true,y_true_KD = y_true[:,:NUM_CLASSES],y_true[:,NUM_CLASSES:]
  y_pred,y_pred_KD = y_pred[:,:NUM_CLASSES],y_pred[:,NUM_CLASSES:]
  # Classic cross-entropy (without temperature)
  CE_loss = logloss(y_true,y_pred)
  # KL-Divergence loss for softened output (with temperature)
  KL_loss = T**2*KLD_Loss(y_true_KD,y_pred_KD)
  
  return lambd*CE_loss + (1-lambd)*KL_loss

def accuracy(y_true,y_pred):
  return categorical_accuracy(y_true,y_pred)

In [21]:
#                                                                OFFLINE KNOWLEDGE DISTILLATION

# Phase 5- END ===>  KTraining the complete STUDENT Model with SOFTMAX layer at the end by using the custom  KD LOSS.

student_model_with_softmax.compile(optimizer='adam',loss=lambda y_true,y_pred: KD_loss(y_true, y_pred,lambd=0.5,T=Temperature),metrics=[accuracy])
myCP = ModelCheckpoint(save_best_only=True,filepath='student_model_trained_regular_kd.h5',monitor = 'val_accuracy')

student_model_with_softmax.fit(x_train,y_train_new,epochs=50,validation_split=0.15,batch_size=128,callbacks=[myCP])

Epoch 1/50
333/333 [==============================] - 2s 6ms/step - loss: 6.2025 - accuracy: 0.2944 - val_loss: 5.3705 - val_accuracy: 0.3236
Epoch 2/50
333/333 [==============================] - 2s 5ms/step - loss: 5.6664 - accuracy: 0.3522 - val_loss: 5.1285 - val_accuracy: 0.3569
Epoch 3/50
333/333 [==============================] - 2s 6ms/step - loss: 5.4905 - accuracy: 0.3712 - val_loss: 4.9655 - val_accuracy: 0.3657
Epoch 4/50
333/333 [==============================] - 2s 6ms/step - loss: 5.3881 - accuracy: 0.3790 - val_loss: 5.2182 - val_accuracy: 0.3509
Epoch 5/50
333/333 [==============================] - 2s 6ms/step - loss: 5.2750 - accuracy: 0.3896 - val_loss: 4.9232 - val_accuracy: 0.3751
Epoch 6/50
333/333 [==============================] - 2s 5ms/step - loss: 5.1849 - accuracy: 0.3963 - val_loss: 5.1598 - val_accuracy: 0.3381
Epoch 7/50
333/333 [==============================] - 2s 5ms/step - loss: 5.1287 - accuracy: 0.4017 - val_loss: 4.6708 - val_accuracy: 0.3987
Epoch 

In [22]:
# STANDALONE student model trained from SCRATCH, ONLY From DATASET
aloneModel = student_generator()
aloneModel.summary()
aloneModel.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

myCP = ModelCheckpoint(_best_only=True,filepath='alone.h5',monitor = 'val_acc')

aloneModel.fit(x_train,y_train,epochs=50,validation_split=0.15,batch_size=128,callbacks=[myCP])

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_10 (Flatten)         (None, 3072)              0         
_________________________________________________________________
dense_20 (Dense)             (None, 64)                196672    
_________________________________________________________________
dense_21 (Dense)             (None, 10)                650       
Total params: 197,322
Trainable params: 197,322
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50
333/333 [==============================] - 2s 5ms/step - loss: 5.7553 - accuracy: 0.0988 - val_loss: 5.3244 - val_accuracy: 0.1015
Epoch 2/50
333/333 [==============================] - 1s 4ms/step - loss: 5.3010 - accuracy: 0.0997 - val_loss: 5.3245 - val_accuracy: 0.1015
Epoch 3/50
333/333 [==============================] - 2s 5ms/step - loss: 5.2993 - accuracy: 

In [ ]:
#                                                             INTERNAL OFFLINE KNOWLEDGE DISTILLATION
# Phase 1 ===>        get the  TARGET layer TEACHER
teacher_model
